In [ ]:
import json
import pandas as pd

In [ ]:
with open('project_transcriptome.json', 'r') as file:
    project_data = json.load(file)

In [ ]:
t_colon_samples = project_data["datasets"]["TCGA-COAD-FPKM"]["groups"]["T-Colon"]["samples"]

In [ ]:
expression_data = pd.read_csv('data/expression_data_tcga_coad_TCGA-COAD_log_fpkm_560_samples_42696_genes.csv', sep=';')
clinical_annotations = pd.read_excel('data/EpiMed_experimental_grouping_2023.07.08_TCGA-COAD(1).xlsx')

In [ ]:
clinical_annotations_filtered = clinical_annotations[clinical_annotations['id_sample'].isin(t_colon_samples)]
clinical_annotations_filtered = clinical_annotations_filtered[['id_sample', 'os_months', 'os_censor']]

In [ ]:
columns_to_keep = ['id_gene', 'gene_symbol'] + t_colon_samples
expression_data_filtered = expression_data[columns_to_keep]

In [ ]:
gec_data = pd.read_csv('data/TCGA-COAD-FPKM_GEC.csv', sep=';')
survival_data = pd.merge(gec_data, clinical_annotations_filtered, on='id_sample', how='inner')
survival_data = survival_data[['id_sample', 'GEC', 'os_months', 'os_censor']]

max_survival_time = 120
survival_data['os_months'] = survival_data['os_months'].apply(lambda x: min(x, max_survival_time))
survival_data['os_censor'] = survival_data.apply(
    lambda row: 0 if row['os_months'] == max_survival_time else row['os_censor'], axis=1
)

In [ ]:
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt
import matplotlib

kmf = KaplanMeierFitter()

plt.figure(figsize=(6, 6))
plt.suptitle('bold figure suptitle', fontsize=14, fontweight='bold')
ax.set_xlabel('xlabel')
ax.set_ylabel('ylabel')

for group in survival_data['GEC'].unique():
    group_data = survival_data[survival_data['GEC'] == group]
    kmf.fit(durations=group_data['os_months'], event_observed=group_data['os_censor'], label=group)
    kmf.plot_survival_function(show_censors = True, ci_show = False)